<a href="https://colab.research.google.com/github/arralia/cs125/blob/main/CS125_Week3_Thursday_ProjectSetup_Data_Indexing_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 125 (Week 3 Thu) — Project Setup + Data + Indexing + Baseline Plan (Jupyter Notebook)

**Goal:** Build a small end-to-end pipeline:

**ingest data → create a logical view → build an index → run queries → baseline ranking**

Use this notebook, fork it and adapt to your project.

**What you should have by the end of this week**
- Project environment ready (repo, task board, shared doc space)
- Defined your data sources (what you will collect + where it comes from)
- Indexing plan draft (what you will store/index; level of granularity)
- Baseline plan (a “minimum working demo” milestone you can hit soon)
- Team checklist + roles


## 0) Setup (Python environment)

Recommended: **JupyterLab** or **Google Colab**.

Minimal deps:
- `pandas`
- `numpy`

If missing: `!pip install pandas numpy`


In [ ]:
import re
import math
import json
from collections import defaultdict, Counter
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import pandas as pd
import numpy as np


## 1) Data ingestion (public dataset)

We’ll use a small public dataset (SMS Spam Collection) because it’s simple, realistic, and great for indexing demos.

If download fails, we fall back to a tiny in-notebook dataset.


In [ ]:
import urllib.request
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

SMS_URL = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
SMS_PATH = DATA_DIR / "sms.tsv"

def download_if_needed(url: str, path: Path) -> bool:
    if path.exists() and path.stat().st_size > 0:
        return True
    try:
        print(f"Downloading: {url}")
        urllib.request.urlretrieve(url, path)
        print(f"Saved to: {path.resolve()}")
        return True
    except Exception as e:
        print("Download failed:", e)
        return False

ok = download_if_needed(SMS_URL, SMS_PATH)

if ok:
    df = pd.read_csv(SMS_PATH, sep="\t", header=None, names=["label", "message"])
    df["doc_id"] = np.arange(len(df))
else:
    df = pd.DataFrame({
        "label": ["ham","spam","ham","ham","spam","ham","spam","ham"],
        "message": [
            "Hey are we still meeting at 3pm?",
            "WINNER!! Claim your prize now by texting WIN to 80085",
            "I'll bring the slides for Thursday discussion.",
            "Can you review my PR before lunch?",
            "URGENT! You have been selected for a free gift card. Reply YES",
            "Let's grab coffee after lecture.",
            "Congratulations you won a lottery, click here to claim",
            "Reminder: project kickoff meeting tomorrow 10am.",
        ]
    })
    df["doc_id"] = np.arange(len(df))

df.head()

,label,message,doc_id
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,2
3,ham,U dun say so early hor... U c already then say...,3
4,ham,"Nah I don't think he goes to usf, he lives aro...",4


## 2) Define a *logical view* (turn raw data into indexable fields)

A logical view is: **how users will search your data**.

For this dataset, we’ll create:
- `tokens`: normalized terms
- `pos`: token → positions (for phrase/proximity)
- metadata: label, length, etc.

In your project, your logical view might be:
- events on a timeline
- locations + activities
- entities extracted from text
- sensor readings aggregated into windows
- JSON fields normalized into columns


In [ ]:
STOPWORDS = set((
    "a an the and or but if then else for of to in on at by with without from as "
    "is are was were be been being this that these those it its im youre we you "
    "they he she them our your their"
).split())

TOKEN_RE = re.compile(r"[a-z0-9]+")

def tokenize(text: str) -> List[str]:
    text = text.lower()
    toks = TOKEN_RE.findall(text)
    toks = [t for t in toks if t not in STOPWORDS]
    return toks

df["tokens"] = df["message"].map(tokenize)
df["length"] = df["tokens"].map(len)

def token_positions(tokens: List[str]) -> Dict[str, List[int]]:
    pos = defaultdict(list)
    for i, t in enumerate(tokens):
        pos[t].append(i)
    return dict(pos)

df["pos"] = df["tokens"].map(token_positions)
df[["doc_id","label","length","message"]].head()

,doc_id,label,length,message
0,0,ham,19,"Go until jurong point, crazy.. Available only ..."
1,1,ham,6,Ok lar... Joking wif u oni...
2,2,spam,28,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,ham,10,U dun say so early hor... U c already then say...
4,4,ham,11,"Nah I don't think he goes to usf, he lives aro..."


## 3) Build an inverted index (postings)

We store postings with:
- `doc_id`
- `tf` (term frequency)
- `positions`

This supports:
- Boolean queries (AND / OR)
- Phrase queries
- TF-IDF baseline ranking


In [ ]:
@dataclass(frozen=True)
class Posting:
    doc_id: int
    tf: int
    positions: Tuple[int, ...]

InvertedIndex = Dict[str, List[Posting]]

def build_inverted_index(df: pd.DataFrame) -> InvertedIndex:
    idx: Dict[str, List[Posting]] = defaultdict(list)
    for row in df.itertuples(index=False):
        doc_id = int(row.doc_id)
        pos_map = row.pos  # dict term -> [positions]
        for term, positions in pos_map.items():
            idx[term].append(Posting(doc_id=doc_id, tf=len(positions), positions=tuple(positions)))
    for term in idx:
        idx[term].sort(key=lambda p: p.doc_id)
    return dict(idx)

inv = build_inverted_index(df)

for t in ["win", "project", "meeting", "free", "lecture"]:
    print(t, "->", inv.get(t, [])[:5])

win -> [Posting(doc_id=2, tf=1, positions=(5,)), Posting(doc_id=11, tf=1, positions=(2,)), Posting(doc_id=134, tf=1, positions=(4,)), Posting(doc_id=312, tf=1, positions=(3,)), Posting(doc_id=335, tf=1, positions=(3,))]
project -> [Posting(doc_id=380, tf=1, positions=(15,)), Posting(doc_id=914, tf=1, positions=(8,)), Posting(doc_id=1228, tf=1, positions=(4,)), Posting(doc_id=1480, tf=1, positions=(25,)), Posting(doc_id=1728, tf=1, positions=(2,))]
meeting -> [Posting(doc_id=57, tf=1, positions=(5,)), Posting(doc_id=364, tf=1, positions=(11,)), Posting(doc_id=493, tf=1, positions=(1,)), Posting(doc_id=590, tf=1, positions=(2,)), Posting(doc_id=914, tf=1, positions=(9,))]
free -> [Posting(doc_id=2, tf=1, positions=(0,)), Posting(doc_id=9, tf=2, positions=(13, 18)), Posting(doc_id=12, tf=1, positions=(5,)), Posting(doc_id=42, tf=2, positions=(9, 12)), Posting(doc_id=56, tf=1, positions=(19,))]
lecture -> [Posting(doc_id=1031, tf=1, positions=(7,)), Posting(doc_id=1270, tf=1, positions=(3,

## 4) Boolean retrieval (baseline #1)

Boolean retrieval is a strong early milestone:
- Easy to implement
- Forces you to define tokens + index granularity
- Gives a working end-to-end demo

We implement:
- `AND` (intersection)
- `OR` (union)


In [ ]:
def postings_docs(term: str) -> List[int]:
    return [p.doc_id for p in inv.get(term, [])]

def and_query(terms: List[str]) -> List[int]:
    sets = [set(postings_docs(t)) for t in terms]
    if not sets:
        return []
    return sorted(set.intersection(*sets))

def or_query(terms: List[str]) -> List[int]:
    out = set()
    for t in terms:
        out |= set(postings_docs(t))
    return sorted(out)

def show_docs(doc_ids: List[int], n: int = 5):
    for doc_id in doc_ids[:n]:
        row = df.loc[df["doc_id"] == doc_id].iloc[0]
        print(f"[{doc_id}] ({row['label']}) {row['message']}")

q1 = ["project", "meeting"]
hits1 = and_query(q1)
print("AND", q1, "->", hits1); show_docs(hits1)

q2 = ["win", "free", "prize"]
hits2 = or_query(q2)
print("\nOR", q2, "->", hits2); show_docs(hits2)


AND ['project', 'meeting'] -> [914]
[914] (ham) Ok lor but not too early. Me still having project meeting now.

OR ['win', 'free', 'prize'] -> [2, 8, 9, 11, 12, 42, 56, 65, 75, 89, 93, 95, 114, 121, 134, 139, 167, 172, 178, 180, 188, 227, 250, 268, 270, 296, 312, 319, 335, 357, 367, 385, 389, 401, 418, 424, 455, 463, 486, 487, 492, 495, 501, 505, 515, 519, 525, 564, 576, 583, 591, 607, 629, 650, 659, 673, 710, 719, 744, 766, 789, 797, 803, 804, 813, 870, 878, 900, 907, 930, 948, 962, 1002, 1007, 1017, 1060, 1067, 1072, 1091, 1137, 1163, 1194, 1205, 1221, 1227, 1229, 1274, 1318, 1364, 1380, 1423, 1444, 1456, 1459, 1463, 1466, 1492, 1506, 1507, 1518, 1521, 1536, 1544, 1574, 1598, 1625, 1653, 1663, 1673, 1688, 1691, 1699, 1718, 1721, 1734, 1759, 1767, 1778, 1780, 1784, 1791, 1793, 1848, 1853, 1874, 1904, 1915, 1929, 1930, 1963, 1970, 1978, 1993, 2003, 2023, 2044, 2064, 2079, 2119, 2121, 2124, 2160, 2189, 2207, 2220, 2225, 2232, 2267, 2290, 2300, 2308, 2309, 2311, 2312, 2313, 2328, 2343, 2

## 5) Phrase queries using positions (baseline #2)

Phrase query example: **"project kickoff"**

Approach:
1) candidate docs must contain *all* terms  
2) verify adjacency via token positions


In [ ]:
def phrase_query(phrase: str) -> List[int]:
    terms = tokenize(phrase)
    if not terms:
        return []
    candidates = and_query(terms)
    if not candidates:
        return []
    results = []
    for doc_id in candidates:
        pos_map = df.loc[df.doc_id == doc_id, "pos"].iloc[0]
        pos_lists = [pos_map.get(t, []) for t in terms]
        base_positions = pos_lists[0]
        offsets = list(range(len(terms)))
        ok = False
        for p in base_positions:
            if all((p + off) in set(pos_lists[i]) for i, off in enumerate(offsets)):
                ok = True
                break
        if ok:
            results.append(doc_id)
    return results

hits = phrase_query("project kickoff")
print("phrase hits:", hits)
show_docs(hits, n=10)


phrase hits: []


## 6) TF-IDF ranking (baseline #3)

Ranked baseline:
- doc vectors of term weights
- query vector
- cosine similarity

Good enough for a Week 3 baseline in many projects.


In [ ]:
N = len(df)
dfreq = {t: len(inv[t]) for t in inv.keys()}

def idf(term: str) -> float:
    return math.log((N + 1) / (dfreq.get(term, 0) + 1)) + 1.0  # smoothed

doc_vecs: List[Dict[str, float]] = [defaultdict(float) for _ in range(N)]
doc_norms = np.zeros(N, dtype=float)

for term, postings in inv.items():
    w_idf = idf(term)
    for p in postings:
        w = (1 + math.log(p.tf)) * w_idf
        doc_vecs[p.doc_id][term] = w

for doc_id in range(N):
    norm = math.sqrt(sum(v*v for v in doc_vecs[doc_id].values()))
    doc_norms[doc_id] = norm if norm > 0 else 1.0

def rank(query: str, top_k: int = 10) -> List[Tuple[int, float]]:
    q_terms = tokenize(query)
    if not q_terms:
        return []
    q_tf = Counter(q_terms)
    q_vec = {t: (1 + math.log(tf)) * idf(t) for t, tf in q_tf.items()}
    q_norm = math.sqrt(sum(v*v for v in q_vec.values())) or 1.0

    scores = []
    for doc_id in range(N):
        dv = doc_vecs[doc_id]
        dot = 0.0
        for term, qw in q_vec.items():
            dw = dv.get(term)
            if dw is not None:
                dot += qw * dw
        score = dot / (q_norm * doc_norms[doc_id])
        if score > 0:
            scores.append((doc_id, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

results = rank("free prize claim now", top_k=8)
results


[(495, np.float64(0.4705918898518977)),
 (1874, np.float64(0.3678994100143857)),
 (2952, np.float64(0.3196365062048202)),
 (4047, np.float64(0.3179192803940924)),
 (3118, np.float64(0.31676402395830877)),
 (4797, np.float64(0.31349731144225773)),
 (576, np.float64(0.3116723800542631)),
 (3529, np.float64(0.31089819226708093))]

In [ ]:
for doc_id, score in results:
    row = df.loc[df.doc_id == doc_id].iloc[0]
    print(f"{score:0.3f}  [{doc_id}] ({row['label']}) {row['message']}")


0.471  [495] (ham) Are you free now?can i call now?
0.368  [1874] (spam) You have WON a guaranteed £1000 cash or a £2000 prize.To claim yr prize call our customer service representative on
0.320  [2952] (ham) Hey now am free you can call me.
0.318  [4047] (spam) Win a £1000 cash prize or a prize worth £5000
0.317  [3118] (ham) Now am free call me pa.
0.313  [4797] (spam) URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893
0.312  [576] (spam) You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327
0.311  [3529] (spam) You are a £1000 winner or Guaranteed Caller Prize, this is our Final attempt to contact you! To Claim Call 09071517866 Now! 150ppmPOBox10183BhamB64XE


## 7) Activity #1 (8–10 min): Improve the logical view (fielded indexing)

**Prompt (Poll Everywhere / in-class):**
1. Add at least **one extra field** to the logical view that helps search.
   - examples: `has_url`, `has_number`, `contains_money`, `contains_email`, `hour_of_day` (if you have timestamps)
2. Use that field in filtering and/or ranking.
3. Show one query that improves.

➡️ Poll Everywhere: each team posts **(a)** the field they added + **(b)** one example query + output.


In [ ]:
# Activity #1 starter: add a couple of simple features.




## 8) Fielded filtering + ranking (realistic baseline pattern)

A practical baseline pattern:
1) retrieve candidates using the index (fast)
2) filter by metadata / fields
3) rank remaining candidates

This is how many production systems work.


## 9) Activity #2 (10–12 min): Implement BM25 (classic IR baseline) OR proximity bonus

Pick ONE:

### Option A — BM25
- Replace TF-IDF with BM25 scoring
- Use `k1=1.2`, `b=0.75`

### Option B — Proximity bonus
- Keep TF-IDF
- Add a bonus if query terms appear close together in the doc (using positions)

➡️ Poll Everywhere: post which option works best


In [ ]:
# Activity #2A (BM25) — ready-to-run baseline (modify/extend)




## 10) Translate this to *your* project (quick worksheet)

### A) Project environment
- Repo: `_____`
- Task board: `_____` (Trello / GitHub Projects / Notion)
- Shared docs: `_____` (Google Drive / Notion)

### B) Data ingestion
- Data sources (≥2):
  1. `_____` (where it comes from, rate, format)
  2. `_____`
- What is manual vs automatic in week 3–4?

### C) Logical view (indexable fields)
- Resource/item type(s): `_____` (documents / events / places / products / actions)
- Fields you will store:
  - primary text: `_____`
  - metadata: `_____`
  - context: `_____`

### D) Indexing plan
- Granularity: document vs paragraph vs event vs window
- Index structures:
  - inverted index over `_____`
  - secondary indexes over metadata fields `_____`

### E) Baseline plan (minimum working demo)
- Input query (typed? contextual? subscription?)
- Output items (≥10 candidate items/actions/resources)
- Baseline you can ship soon:
  - retrieval: boolean or TF-IDF or BM25
  - ranking: `_____`
  - UI/prototype: `_____`


## 11) Optional: export your index (so a teammate can use it)

In real projects, one teammate may build the index while another builds the UI.
Export a lightweight artifact (JSON) for collaboration.


In [ ]:
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(exist_ok=True)

def export_index(inv: InvertedIndex, path: Path, max_terms: int = 5000):
    items = {}
    for i, (term, postings) in enumerate(inv.items()):
        if i >= max_terms:
            break
        items[term] = [{"doc_id": p.doc_id, "tf": p.tf, "pos": list(p.positions)} for p in postings]
    with open(path, "w", encoding="utf-8") as f:
        json.dump(items, f)
    print("Wrote:", path.resolve())

export_index(inv, EXPORT_DIR / "inverted_index.json")


Wrote: /content/export/inverted_index.json


## 12) Checklist to take away from this session:

- [ ] Repo created + all teammates have access
- [ ] Task board created + first 10 tasks added
- [ ] Shared doc space created
- [ ] Data sources listed (what + where + format)
- [ ] Logical view drafted (fields)
- [ ] Index plan drafted (what is indexed; granularity)
- [ ] Query environment defined (typed/context/subscription)
- [ ] At least 10 candidate items/actions/resources listed
- [ ] Baseline demo milestone defined (date + scope)
- [ ] Roles assigned (data ingestion / indexing / UI / testing+demo)
